In [1]:
-- basic operands
a = torch.Tensor(5, 3)  -- construct a 5x3 matrix unintialized
print(a)

0.000000 0.000000 0.000000
nan 0.000000 0.000000
0.000000 nan 0.000000
0.000000 0.000000 nan
0.000000 0.000000 0.000000
[torch.DoubleTensor of size 5x3]



In [2]:
a = torch.rand(5, 3)
print(a)

 0.5831  0.7915  0.0417
 0.8352  0.9839  0.7909
 0.8207  0.0156  0.4927
 0.9600  0.0892  0.6052
 0.9964  0.4888  0.8902
[torch.DoubleTensor of size 5x3]



In [3]:
b = torch.rand(3, 4)

In [4]:
-- three ways of doing multiplication
a*b

 1.1365  0.7627  0.7749  0.2538
 1.6202  1.6921  1.6799  0.6308
 0.6406  0.6566  0.6222  0.2361
 0.8217  0.8507  0.8119  0.3081
 1.2755  1.4422  1.4064  0.5402
[torch.DoubleTensor of size 5x4]



In [5]:
torch.mm(a, b)

 1.1365  0.7627  0.7749  0.2538
 1.6202  1.6921  1.6799  0.6308
 0.6406  0.6566  0.6222  0.2361
 0.8217  0.8507  0.8119  0.3081
 1.2755  1.4422  1.4064  0.5402
[torch.DoubleTensor of size 5x4]



In [7]:
c = torch.zeros(5, 4)
c:mm(a, b)

In [8]:
print(c)

 1.1365  0.7627  0.7749  0.2538
 1.6202  1.6921  1.6799  0.6308
 0.6406  0.6566  0.6222  0.2361
 0.8217  0.8507  0.8119  0.3081
 1.2755  1.4422  1.4064  0.5402
[torch.DoubleTensor of size 5x4]



In [8]:
--- concatenation
torch.cat(torch.ones(3), torch.zeros(2))

 1
 1
 1
 0
 0
[torch.DoubleTensor of size 5]



In [9]:
torch.cat(torch.ones(3, 2), torch.zeros(2, 2), 1)

 1  1
 1  1
 1  1
 0  0
 0  0
[torch.DoubleTensor of size 5x2]



In [9]:
torch.cat(torch.ones(3, 2), torch.zeros(3, 3), 2)

 1  1  0  0  0
 1  1  0  0  0
 1  1  0  0  0
[torch.DoubleTensor of size 3x5]



In [10]:
-- neural networks
-- linear module
require 'nn';
lin = nn.Linear(5, 3)

In [11]:
lin

nn.Linear(5 -> 3)
{
  gradBias : DoubleTensor - size: 3
  weight : DoubleTensor - size: 3x5
  _type : torch.DoubleTensor
  output : DoubleTensor - empty
  gradInput : DoubleTensor - empty
  bias : DoubleTensor - size: 3
  gradWeight : DoubleTensor - size: 3x5
}


In [12]:
lin['weight']

-0.1637 -0.1335 -0.0658 -0.4250 -0.3603
-0.2468 -0.2617 -0.1374  0.4378  0.0821
-0.2458  0.0627  0.2897  0.1919 -0.3562
[torch.DoubleTensor of size 3x5]



In [13]:
-- forward pass
x = torch.rand(5)
print(x)

 0.6013
 0.1463
 0.6839
 0.0133
 0.9836
[torch.DoubleTensor of size 5]



In [14]:
y = lin:forward(x)
print(y)

-0.4579
-0.5157
-0.6395
[torch.DoubleTensor of size 3]



In [15]:
lin.weight * x + lin.bias

-0.4579
-0.5157
-0.6395
[torch.DoubleTensor of size 3]



In [16]:
-- backward pass
lin:zeroGradParameters()
print(lin.gradWeight)

 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
[torch.DoubleTensor of size 3x5]



In [17]:
grad = torch.ones(3):mul(-1)
grad[2] = 1
print(grad)

-1
 1
-1
[torch.DoubleTensor of size 3]



In [18]:
lin:backward(x, grad)

 0.1627
-0.1909
-0.3614
 0.6709
 0.7985
[torch.DoubleTensor of size 5]



In [19]:
print(lin.gradWeight)

-0.6013 -0.1463 -0.6839 -0.0133 -0.9836
 0.6013  0.1463  0.6839  0.0133  0.9836
-0.6013 -0.1463 -0.6839 -0.0133 -0.9836
[torch.DoubleTensor of size 3x5]



In [20]:
-- manual gradient descent
lin.weight:add(0.1*lin.gradWeight)
lin.bias:add(0.1*lin.gradBias)

In [21]:
lin:forward(x)

-0.7398
-0.2338
-0.9213
[torch.DoubleTensor of size 3]



In [23]:
-- more complicated networks
-- with package 'nn'
net = nn.Sequential();
net:add(nn.Linear(20, 10));
net:add(nn.Tanh());
net:add(nn.Linear(10, 10));
net:add(nn.Tanh());
net:add(nn.Linear(10, 1));

In [24]:
x = torch.rand(20)
y1 = net:forward(x)
print(y1)

-0.2877
[torch.DoubleTensor of size 1]



In [23]:
-- with package 'nngraph'
require 'nngraph';
g1 = - nn.Linear(20, 10)

g2 = g1
   - nn.Tanh()
   - nn.Linear(10, 10)
   - nn.Tanh()
   - nn.Linear(10, 1)
gnet = nn.gModule({g1}, {g2})

In [24]:
-- fancy network
input = - nn.Identity()
L1 = input
   - nn.Linear(10, 20)
   - nn.Tanh()
L2 = {input, L1}
   - nn.JoinTable(1)
   - nn.Linear(30, 60)
   - nn.Tanh()
L3 = {L1, L2}
   - nn.JoinTable(1)
   - nn.Linear(80, 1)
   - nn.Tanh()
g = nn.gModule({input},{L3})

In [25]:
require 'pretty-nn'
graph.dot(g.fg, 'fancy', 'fancy')

In [28]:
-- training --
-- load serialized preprocessed data
xx, yy = unpack(torch.load("redwine-quality.torch"))
print(xx:size())
print(yy:size())

 1599
   11
[torch.LongStorage of size 2]

 1599
[torch.LongStorage of size 1]



In [26]:
torch.manualSeed(1234)
-- build the network
g1 = - nn.Linear(11, 11)
g2 = g1
   - nn.Tanh()
   - nn.Linear(11, 11)
   - nn.Tanh()
   - nn.Linear(11, 1)
winenet = nn.gModule({g1}, {g2})
-- mean square error
loss = nn.MSECriterion()

In [31]:
-- train!
require 'optim';
n_epoches = 10
n_examples = xx:size(1)

W, gradW = winenet:getParameters()
optimState = {}

for epoch = 1, n_epoches do
   local total_loss = 0
   for i=1, n_examples do
     x = xx[i]
     y = torch.Tensor({yy[i]})
     winenet:zeroGradParameters()
     function feval()
        local predicted = winenet:forward(x)
        local L = loss:forward(predicted, y)
        total_loss = total_loss + L
        local dL_dy = loss:backward(predicted, y)
        winenet:backward(x, dL_dy) -- computes and updates gradW
        return L, gradW
     end
     optim.adadelta(feval, W, optimState)
   end
   print('at epoch', epoch, 'avg loss', total_loss/n_examples)
end

at epoch	1	avg loss	0.0045061651022055	


at epoch	2	avg loss	0.0044699241016204	


at epoch	3	avg loss	0.0044438207931137	


at epoch	4	avg loss	0.0044207669711346	


at epoch	5	avg loss	0.0043997806790086	


at epoch	6	avg loss	0.004380264388858	


at epoch	7	avg loss	0.0043608064125197	


at epoch	8	avg loss	0.0043431585576826	


at epoch	9	avg loss	0.00432850799651	


at epoch	10	avg loss	0.004315805457268	


In [32]:
-- RNN
-- stepwise function
ht1 = - nn.Identity()
xt = - nn.Identity()
ht = {ht1, xt} 
   - nn.JoinTable(1)
   - nn.Linear(20, 10)
   - nn.Tanh()
stepfunction = nn.gModule({ht1, xt}, {ht})

In [33]:
-- build clones

function share_params(cell, src)
  if torch.type(cell) == 'nn.gModule' then
    for i = 1, #cell.forwardnodes do
      local node = cell.forwardnodes[i]
      if node.data.module then
        node.data.module:share(src.forwardnodes[i].data.module,
                               'weight', 'bias', 'gradWeight', 'gradBias')
      end
    end
  elseif torch.isTypeOf(cell, 'nn.Module') then
    cell:share(src, 'weight', 'bias', 'gradWeight', 'gradBias')
  else
    error('parameters cannot be shared for this input')
  end
end

-- getParameters() must be called before cloning, 
-- since getParameters() function will reallocate memory for W and gradW
-- the pointers in clones created before getParameters() will not be valid
-- after calling this function
W, gradW = stepfunction:getParameters()

clones = {}
N = 100
for i = 1, N do
    clones[i] = stepfunction:clone()
    share_params(clones[i], stepfunction)
end

In [34]:
print(clones)

{
  1 : 
    nn.gModule
    {
      bg : 
        graph.Graph
        {
          nodes : 
            {
              1 : 
                nngraph.Node
                {
                  data : table: 0x10034778
                  visited : false
                  id : 1
                  children : table: 0x0d8c9c18
                  marked : false
                }
              2 : 
                nngraph.Node
                {
                  data : table: 0x100349e0
                  visited : false


                  id : 2
                  children : table: 0x0d54e8e8
                  marked : false
                }
              3 : 
                nngraph.Node
                {
                  data : table: 0x0ff01e18
                  visited : false
                  id : 3
                  children : table: 0x101467d8
                  marked : false
                }
              4 : 
                nngraph.Node
                {
                  data : table: 0x10146930
                  visited : false
                  id : 4
                  children : table: 0x101376c8
                  marked : false
                }
              5 : 
                nngraph.Node
                {
                  data : table: 0x0ffdbaa0
                  visited : false
                  id : 5
                  children : table: 0x10137250
                  marked : false
                }
              6 : 
                nngraph.Node
                {
             

                 marked : false
                }
              7 : 
                nngraph.Node
                {
                  data : table: 0x0ffdbb90
                  visited : false
                  id : 7
                  children : table: 0x100eba80
                  marked : false
                }
              8 : 
                nngraph.Node
                {
                  data : table: 0x10085f40
                  visited : false
                  id : 8
                  children : table: 0x100eb668
                  marked : false
                }
              9 : 
                nngraph.Node
                {
                  data : table: 0x100bb930
                  visited : false
                  id : 9
                  children : table: 0x0d54e6d8
                  marked : false
                }
              10 : 
                nngraph.Node
                {
                  data : table: 0x100bbab8
                  visited : false
        


              nngraph.Node : 1
              nngraph.Node : 5
              nngraph.Node : 6
              nngraph.Node : 4
              nngraph.Node : 7
              nngraph.Node : 3
              nngraph.Node : 8
              nngraph.Node : 10
            }
          edges : 
            {
              1 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              2 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              3 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              4 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              5 : 
                graph.Edge
                {
               

      from : nngraph.Node
                }
              8 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              9 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              10 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              graph.Edge : 10
              graph.Edge : 9
              graph.Edge : 8
              graph.Edge : 7
              graph.Edge : 5
              graph.Edge : 3
              graph.Edge : 6
              graph.Edge : 2
              graph.Edge : 1
              graph.Edge : 4
            }
        }
      forwardnodes : 
        {
          1 : 
            nngraph.Node
            {
              data : 
                {
                  annotations : table: 0x100bb

...

  from : nngraph.Node
                }
              7 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              8 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              9 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              10 : 
                graph.Edge
                {
                  to : nngraph.Node
                  from : nngraph.Node
                }
              graph.Edge : 4
              graph.Edge : 7
              graph.Edge : 6
              graph.Edge : 5
              graph.Edge : 3
              graph.Edge : 1
              graph.Edge : 2
              graph.Edge : 10
              graph.Edge : 9
              graph.Edge : 8
            }
        }
      innode : 
    


            {
              data : 
                {
                  annotations : table: 0x100bf628
                  input : table: 0x0da6cff8
                  mapindex : table: 0x0d540098
                  reverseMap : table: 0x103529e8
                  forwardNodeId : 2
                }
              visited : false
              id : 1
              children : 
                {
                  1 : nngraph.Node
                  nngraph.Node : 1
                }
              marked : false
            }
          2 : 
            nngraph.Node
            {
              data : 
                {
                  annotations : table: 0x0d878430
                  module : nn.Tanh
                  reverseMap : table: 0x0d9bc640
                  forwardNodeId : 1
                  mapindex : table: 0x0d8ca6f8
                }
              visited : false
              id : 2
              children : 
                {
                  1 : nngraph.Node
                

   marked : false
            }
          4 : 
            nngraph.Node
            {
              data : 
                {
                  annotations : table: 0x1000adf8
                  module : nn.JoinTable
                  reverseMap : table: 0x0ff39fc8
                  forwardNodeId : 4
                  mapindex : table: 0x0d87bcc0
                }
              visited : false
              id : 4
              children : 
                {
                  1 : nngraph.Node
                  2 : nngraph.Node
                  nngraph.Node : 2
                  nngraph.Node : 1
                }
              marked : false
            }
          5 : 
            nngraph.Node
            {
              data : 
                {
                  annotations : table: 0x10145638
                  module : nn.Identity
                  reverseMap : table: 0x0d5b1588
                  forwardNodeId : 5
                  mapindex : table: 0x0ff68fe8
                }
     

          {
              data : 
                {
                  annotations : table: 0x10351e30
                  module : nn.Identity
                  reverseMap : table: 0x103521a0
                  forwardNodeId : 6
                  mapindex : table: 0x10352218
                }
              visited : false
              id : 6
              children : 
                {
                  1 : nngraph.Node
                  nngraph.Node : 1
                }
              marked : false
            }
          8 : 
            nngraph.Node
            {
              data : 
                {
                  annotations : table: 0x0ff09128
                  input : table: 0x100c50b8
                  mapindex : table: 0x100aa480
                  selectindex : 2
                  forwardNodeId : 8
                  reverseMap : table: 0x10351cc8
                }
              visited : false
              id : 8
              children : 
                {
                

  {
              annotations : 
                {
                  _debugLabel : [[C]]:-1_
                }
              input : table: 0x0da6cff8
              mapindex : 
                {
                  1 : table: 0x0da5f9a8
                  table: 0x0da5f9a8 : 1
                }
              reverseMap : table: 0x103529e8
              forwardNodeId : 2
            }
          visited : false
          id : 1
          children : 
            {
              1 : 
                nngraph.Node
                {
                  data : table: 0x0da5f9a8
                  visited : false
                  id : 2
                  children : table: 0x1014f920
                  marked : false
                }
              nngraph.Node : 1
            }
          marked : false
        }
      nInputs : 2
      verbose : false
    }
}


In [35]:
n = 7
h0 = torch.rand(10)
x = torch.rand(n, 10)
h = torch.zeros(n+1, 10)

W:uniform(-1, 1)

-- forward
h[1] = h0
for i = 1, n do
    h[i+1] = clones[i]:forward{h[i], x[i]}
end

In [36]:
print(h)

 0.5982  0.7899  0.1065  0.3861  0.2972  0.9791  0.0584  0.2194  0.6213  0.0929
-0.8879  0.9448 -0.9973  0.2613 -0.6036  0.8370  0.9533  0.2834 -0.9841 -0.9885
-0.9585 -0.8517 -0.9440  0.9861  0.7925  0.9844 -0.4312  0.9621 -0.9957 -0.7714
-0.9906 -0.9941 -0.6195  0.9659 -0.7310  0.4731  0.5748 -0.6886 -0.9582 -0.9616
 0.2694 -0.9459  0.4245  0.9859 -0.8624  0.9984 -0.4313  0.6631 -0.6663 -0.9247
-0.9560 -0.2291  0.7429  0.7207  0.8395  0.8807  0.7682 -0.9345 -0.9656 -0.9701
-0.6979 -0.8491 -0.9684  0.5480 -0.4236  0.9153  0.9843 -0.5267 -0.4332 -0.9978
-0.4794 -0.7863 -0.6081  0.9659  0.3993  0.9734 -0.2141  0.8281 -0.9948 -0.8532
[torch.DoubleTensor of size 8x10]



In [37]:
-- before backward
-- clean grad weights
stepfunction:zeroGradParameters()
print(gradW:norm())

0	


In [38]:
-- backward

grad_h = torch.zeros(h:size())
grad_h[n+1] = torch.rand(10)
for i = n, 1, -1 do
    local grads = clones[i]:backward({h[i], x[i]}, grad_h[i+1])
    grad_h[i], grad_xi = unpack(grads)
end

In [39]:
print(grad_h)

 0.0485 -0.0076 -0.1426 -0.1481  0.2058 -0.3213  0.0837 -0.1005  0.1704 -0.0184
 0.1845 -0.0528 -0.0360 -0.1501 -0.2151  0.0847  0.2186  0.1488 -0.0878  0.0359
-0.2914  0.2014  0.1656  0.1997  0.3198  0.1600 -0.1599  0.0092  0.1554  0.3294
 0.2680 -0.1166 -0.4199 -0.0612 -0.2679 -0.1568  0.1995 -0.0984  0.0512 -0.2923
 0.0193  0.0260  0.4646 -0.0356 -0.1129  0.4859 -0.0190  0.4001 -0.4585  0.2868
-0.0027  0.0779 -0.3206 -0.1302  1.1846 -1.5339 -0.7444 -0.8610  0.8834 -0.2304
-0.1865  0.0984  0.7338 -0.7272 -1.1351  0.1258  1.1297  0.3235  0.4508  0.2306
 0.8840  0.9886  0.5671  0.0696  0.9945  0.0574  0.1787  0.8679  0.0122  0.8638
[torch.DoubleTensor of size 8x10]



In [40]:
print(gradW:norm())

5.4113012786391	
